15/03/2025

In [1]:
import pandas as pd

In [2]:
df_clienti = pd.read_csv(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\clienti.csv",sep = ";")


### Full text creation

In [3]:
import ollama
import json
import os

In [4]:
def create_full_text(row, prefix="search_document"):
    return f"{prefix}: " + ", ".join(
        f"{col}: {val}"
        for col, val in row.items() 
        if pd.notna(val)
    )

df_clienti["full_text"] = df_clienti.apply(create_full_text, axis=1)

In [25]:
df_clienti_full_text = df_clienti["full_text"]

### Embedding generation

NB: sulla repo di nomic-embed-text dicono che bisogna mettere una chiave per dire che tipo di task si vuole fare.

In [5]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")

In [23]:
# import google.generativeai as genai

# genai.configure(api_key=API_KEY)

# models = genai.list_models()

# for model in models:
#     print(model.name)


In [6]:
import os
import pandas as pd
import ollama
import google.generativeai as genai
import time

def get_embedding(text, method="local", task_type = "retrieval_document"):
    if method == "local":
        try:
            response = ollama.embeddings(model="nomic-embed-text", prompt=text)
            return response["embedding"]
        except Exception as e:
            print(f"Errore durante l'embedding: {e}")
            return None
    if method == "api":
        try:
            #time.sleep(2)
            response = genai.embed_content(
                model="models/text-embedding-004",
                content=text,
                task_type=task_type
            )
            return response["embedding"]
            
        except Exception as e:
            print(f"Errore durante l'embedding: {e}")
            return None

path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\nomic_search_index.parquet"

if not os.path.exists(path):
    df_clienti_emb = df_clienti_full_text.apply(get_embedding, method="api")
    df_clienti_emb = pd.DataFrame(df_clienti_emb)
    df_clienti_emb.to_parquet(path, engine="pyarrow")
    print("Embeddings salvati")
else:
    print(f"Il path '{path}' esiste già, uscita dal ciclo.")


Il path 'C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\nomic_search_index.parquet' esiste già, uscita dal ciclo.


In [7]:
df_clienti_emb = pd.read_parquet(path)

ci ha messo 13 min x 6600 clienti.

la dimensione è di 768

### OCR:

con gemini flash

In [65]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")
doc_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\inputs\sunflower.pdf"
with open(doc_path, "rb") as doc_file:
    doc_data = base64.standard_b64encode(doc_file.read()).decode("utf-8")
prompt = """
Trascrivi fedelmente il contenuto di questo documento PDF. Non aggiungere spiegazioni, riassunti o commenti.
Devi solo riportare il testo esattamente come appare nel documento.
"""
response = model.generate_content([
    {'mime_type': 'application/pdf', 'data': doc_data},
    prompt
])
#print(response.text)
ordine = response.text


In [66]:
ordine

"ordini_in@zoppis.com\nDa:\nInviato:\nA:\nOggetto:\nzoppis@zoppis.com\nvenerdi 13 dicembre 2024 14:21\nordini_in@zoppis.com\nNuovo ordine on-line\n****** Nuovo ordine da Internet ******§\nValOrd:#2024_12_13#1421081*14285\nIDOrdine:20010057035\nCodUtente:270002805\nData:13/12/2024 14:20:385\nStato:Da preparareş\nRagSoc_Dest:SUNFLOWER SRL§\nIndirizzo Dest: VIA BORGO VICO 395\nCAP_Dest:221005\nComune_Dest:COMO§\nProv_Dest:CO§\nPaese_Dest:Italia§\nNumTel_Dest:3936886595§\nNumCellulare_Dest:39368865955\nPersona_Dest:GIUSEPPES\nNumFax_Dest:§\nPagamento:Contrassegnos\nTipoConsegna:Normales\nNote: DA CONSEGNARE MARTEDI' 17/12 CON IL RESTO DELL'ORDINE CHE VI MANDERANNO !!! GRAZIE MILLES\nTipoOrdine:Normales\nMessaggio:§\nDocum:15\nIDContatto:2085\nInviato:Vero§\nTotOrdinelvalncl:49,9595\nTotPesoLordo:1,52§\nSpese TrasportolvaEscl:05\nSpeselmballaggiolvaEscl:05\nSpeseAssicurazionelva Escl:0§\nSpese ConsRapidalvaEscl:05\n####>>>>RigheOrdine<<<<####§\nDProdotto | NomeProdottoCompleto | Quantita | 

In [73]:
# ordine = "ORDINE per il cliente MOAM di stresa, golf club alpino 13/12/24. ristorante Paolo Boneschi. Milano\nMittente Erik Calati <erikcalati@gmail.com>\nDestinatario Zoppis Srl-ordini <ordini@zoppis.com>\nCc <flavio.allaria@gmail.com>\nData 2024-12-13 18:14\n\nBuona sera in allegato l'ordine in oggetto:\n• 2 box asahi 330 ml\n• 3box nastro azzurro 330 ml\n• 3 box Thomas Henry pompelmo rosa\n• 1 box coca cola 200 ml\n• 1 box coca cola 0 200 ml\n• 1 box Espolon Blanco\n• 1 box Campari\n• 2 bt Maraschino\n• 1 Punt e mes\n• 1 Ketel One Vodka\n• 1 box achillea arancia 200 ml\n• 1 box achillea pera 200 ml\n\nGrazie, Erik\n"
# ordine = "ORDINE per il cliente MOAM di stresa, golf club alpino 13/12/24. ristorante Paolo Boneschi. Milano\nMittente Erik Calati <erikcalati@gmail.com>\nDestinatario Zoppis Srl-ordini <ordini@zoppis.com>\nCc <flavio.allaria@gmail.com>\nData 2024-12-13 18:14\n\nBuona sera in allegato l'ordine in oggetto:\n• 2 box asahi 330 ml\n• 3box nastro azzurro 330 ml\n• 3 box Thomas Henry pompelmo rosa\n• 1 box coca cola 200 ml\n• 1 box coca cola 0 200 ml\n• 1 box Espolon Blanco\n• 1 box Campari\n• 2 bt Maraschino\n• 1 Punt e mes\n• 1 Ketel One Vodka\n• 1 box achillea arancia 200 ml\n• 1 box achillea pera 200 ml\n\nGrazie, Erik\n"
# ordine = "BOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nNumero\n230032104\nP.IVA/Cod. fisc,\nIT01576310039\nData\n13/12/2024\nVs. Riferimento\nValuta\nEUR\nAgente\nPag.\n1 of 1\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\nCod. articolo\nDescrizione articolo\nQta U.M. Prezzo netto Cod. IVA Imponibile\n19675\nACQUA BRILLANTE TONICA VAP\n1.00 CT\n11.95\n22.00\n14.58\n15061\nCACHACA LEBLON 70 MAISON LEBLON\n6.00 CT\n603.00\n22.00\n735.66\n8729\nGIN HENDRICK'S CL100\n2.00 PZ\n65.90\n22.00\n80.40\n15666\nHIERBAS DE IBIZA Litro AROMATICAS DE IBIZA!\n12.00 PZ\n203.40\n22.00\n248.15\n24626\nSAMBUCA RAMAZZOTI CL70\n3.00 PZ\n20.85\n22.00\n25.44\n24945\nSHARIS FELLUGA VENEZIE\n30.00 PZ\n289.50\n22.00\n353.19\n13035\nTEQUILA PATRON Silver CL70\n2.00 PZ\n67.50\n22.00\n82.35\n20577\nVODKA 42 BELOW LITRO\n30.00 PZ\n436.50\n22.00\n532.53\n"
# ordine = "BOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nNumero\n230032104\nP.IVA/Cod. fisc,\nIT01576310039\nData\n13/12/2024\nVs. Riferimento\nValuta\nEUR\nAgente\nPag.\n1 of 1\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\nCod. articolo\nDescrizione articolo\nQta U.M. Prezzo netto Cod. IVA Imponibile\n19675\nACQUA BRILLANTE TONICA VAP\n1.00 CT\n11.95\n22.00\n14.58\n15061\nCACHACA LEBLON 70 MAISON LEBLON\n6.00 CT\n603.00\n22.00\n735.66\n8729\nGIN HENDRICK'S CL100\n2.00 PZ\n65.90\n22.00\n80.40\n15666\nHIERBAS DE IBIZA Litro AROMATICAS DE IBIZA!\n12.00 PZ\n203.40\n22.00\n248.15\n24626\nSAMBUCA RAMAZZOTI CL70\n3.00 PZ\n20.85\n22.00\n25.44\n24945\nSHARIS FELLUGA VENEZIE\n30.00 PZ\n289.50\n22.00\n353.19\n13035\nTEQUILA PATRON Silver CL70\n2.00 PZ\n67.50\n22.00\n82.35\n20577\nVODKA 42 BELOW LITRO\n30.00 PZ\n436.50\n22.00\n532.53\n"
ordine = "nData:13/12/2024 14:20:385\nStato:Da preparareş\nRagSoc_Dest:SUNFLOWER SRL\nIndirizzo Dest: VIA BORGO VICO 395\nCAP_Dest:221005\nComune_Dest:COMO§\nProv_Dest:CO§\nPaese_Dest:Italia§\nNumTel_Dest:3936886595§\nNumCellulare_Dest:39368865955\nPersona_Dest:GIUSEPPES\nNumFax_Dest:§\nPagamento:Contrassegnos\nTipoConsegna:Normales\nNote: DA CONSEGNARE MARTEDI' 17/12 CON IL RESTO DELL'ORDINE CHE VI MANDERANNO !!! GRAZIE MILLES\nTipoOrdine:Normales\nMessaggio:§\nDocum:15\nIDContatto:2085\nInviato:Vero§\nTotOrdinelvalncl:49,9595\nTotPesoLordo:1,52§\nSpese TrasportolvaEscl:05\nSpeselmballaggiolvaEscl:05\nSpeseAssicurazionelva Escl:0§\nSpese ConsRapidalvaEscl:05\n####>>>>RigheOrdine<<<<####§\nDProdotto | NomeProdottoCompleto | Quantita | LivSconto | Prezzo | Offertaş\n18361|GIN KI NO BI KYOTO DRY GIN, 0, 0, 70|1|0|40,95| Falso§\n***FINEORDINE***§\n"



### RAG

In [57]:
path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\nomic_search_index.parquet"
df_clienti_emb = pd.read_parquet(path)

In [74]:
query = "search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine: \n" + ordine
query_emb = get_embedding(query,task_type="retrieval_query",method="local")

In [75]:
import numpy as np
from scipy.spatial.distance import cdist
k = 100
embeddings_series = df_clienti_emb["full_text"]
query_embedding = query_emb  
embedding_matrix = np.array(embeddings_series.tolist(), dtype=np.float32)
print("Shape della matrice degli embedding:", embedding_matrix.shape)
print("Shape dell'embedding della query:", len(query_embedding))
similarities = 1 - cdist([query_embedding], embedding_matrix, metric="cosine")[0]
top_k_indices = np.argsort(similarities)[::-1][:k]
print("Top k Indici più simili:", top_k_indices)
print("Similarità corrispondenti:", similarities[top_k_indices])

rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())

Shape della matrice degli embedding: (6620, 768)
Shape dell'embedding della query: 768
Top k Indici più simili: [4264  728 3595 3678 6219 6171  259 2529 4212 3451 1276 4016  863 4022
 2788 1704 4295   42 5655 2480   90 4010 5389 5836 4574 1190 1142 2091
 3721 5033 2976 5799   78 6051 3150 1947 2706  676 2446 3308  536 2288
 1187 1587 4741  709 4048 4236  803 4502  425  967 4013 1685 4996 2922
 3548 6241 4863  294 1318 3233  104 3755 3238 3130 4939 5403 2418 3244
 4059 3181 5838 6585 6523 1663  646 5692 4504 4715 3627  956 4397 2574
 1189 1238 2473 1385 4781 2866 6072  853 3587 4512 2549 2614 5994 5852
 3275 3043]
Similarità corrispondenti: [0.77674715 0.77136293 0.77064446 0.76960013 0.76675216 0.766708
 0.76632928 0.76481963 0.76333173 0.76267658 0.76261844 0.76228274
 0.76205494 0.76132369 0.76084531 0.76067765 0.76061716 0.76013039
 0.75983874 0.75977646 0.75974411 0.7596456  0.75926386 0.75889156
 0.7581871  0.75812721 0.75805054 0.75798922 0.75797092 0.75791978
 0.75788632 0.75780

In [76]:
rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())
prompt_rag = f"""search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

{ordine}

 **IMPORTANTE: FINE ORDINE**

 "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto alla destinazione dell'ordine e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n"
  
 🚀 **IMPORTANTE: INIZIO ELENCO DATI ESTRATTI TRAMITE RAG** 🚀
 
 {rag_results}

  **IMPORTANTE: FINE ELENCO DATI ESTRATTI TRAMITE RAG**

"\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest in formato JSON, non voglio alcun tipo di dettaglio aggiuntivo"" 
"""

In [70]:
print(prompt_rag)

search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

ordini_in@zoppis.com
Da:
Inviato:
A:
Oggetto:
zoppis@zoppis.com
venerdi 13 dicembre 2024 14:21
ordini_in@zoppis.com
Nuovo ordine on-line
****** Nuovo ordine da Internet ******§
ValOrd:#2024_12_13#1421081*14285
IDOrdine:20010057035
CodUtente:270002805
Data:13/12/2024 14:20:385
Stato:Da preparareş
RagSoc_Dest:SUNFLOWER SRL§
Indirizzo Dest: VIA BORGO VICO 395
CAP_Dest:221005
Comune_Dest:COMO§
Prov_Dest:CO§
Paese_Dest:Italia§
NumTel_Dest:3936886595§
NumCellulare_Dest:39368865955
Persona_Dest:GIUSEPPES
NumFax_Dest:§
Pagamento:Contrassegnos
TipoConsegna:Normales
Note: DA CONSEGNARE MARTEDI' 17/12 CON IL RESTO DELL'ORDINE CHE VI MANDERANNO !!! GRAZIE MILLES
TipoOrdine:Normales
Messaggio:§
Docum:15
IDContatto:2085
Inviato:Vero§
TotOrdinelvalncl:49,9595
TotPesoLordo:1,52§
Spese TrasportolvaEscl:05
SpeselmballaggiolvaEscl:05
SpeseAssicurazionelva Escl:0§
Spese ConsRapidalvaE

In [77]:
def count_tokens_simple(text):
    return len(text.split())

print(count_tokens_simple(prompt_rag))


4230


In [78]:
generation_config = {
  "temperature": 0.1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json", # posso richiedere esplicitamente il formato strutturato della risposta
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
    #system_instruction="Sei un venditore, quindi i campi vuoti non li sostituisci con Null ma con valori inventati!" # solo come esperimento..
)

output_rag = model.generate_content(prompt_rag)
print(output_rag.text)

[
  {
    "IDCliente": "8443",
    "RagSoc": "SUNFLOWER SRL - BISTROT",
    "Indirizzo_Dest": "VIA BORGOVICO, 39/A"
  }
]
